In [68]:
from python_developer import PythonDeveloperCP
import os
import openai
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field
import json
import re
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import CommaSeparatedListOutputParser, PydanticOutputParser
import csv
from datetime import datetime
import numpy as np

SESSION_TIME_LIMIT = 60  # minutes


class StepInfo(BaseModel):
    question_summary: str = Field(description="summary of the question in very few words")
    ai_comment: str = Field(description="comment on the user's answer")
    score: int = Field(description="score of the user's answer")
    new_question: str = Field(description="any new question")
    further_questions: str = Field(
        description="the only word CONTINUE if it's needed to propose further questions on the topic, else the only word STOP")


class FinalInfo(BaseModel):
    ai_comment: str = Field(description="comment on the user's answer")
    score: int = Field(description="score of the user's answer")


class AISkiller:
    def __init__(self, student_name='Mario', carrier_path=PythonDeveloperCP(), localization="italian"):
        self.step_info_parser = PydanticOutputParser(pydantic_object=StepInfo)
        self.final_info_parser = PydanticOutputParser(pydantic_object=FinalInfo)
        self.carrier_path = carrier_path
        self.student_name = student_name
        self.models = {
            "question_generator": ChatOpenAI(model="gpt-4-0613", temperature=.75,
                                             openai_api_key=os.getenv("openai_key"), max_tokens=500),
            "evaluator": ChatOpenAI(model="gpt-4-0613", temperature=0., openai_api_key=os.getenv("openai_key"),
                                    max_tokens=500),
        }
        self.session_ts = datetime.now()
        self.localization = localization
        self.assessment_session_started = False

    def get_skill_to_evaluate(self):
        """
        Return a skill to evaluate, randomly selected from each module in the
        Carrier Path [self.carrier_path]

        Returns:
            Dict{"area": Skill, "skill": Skill}:    A dictionary of the randomly 
                                                    selected skill from each module.
        """
        skills = [{"area": area, "skill": skill} for (area, skill) in self.carrier_path.get_skills_to_evaluate()]
        return np.random.choice(skills)

    def get_question(self, student_name, carrier_path_name, area_name, skill):
        """
        Generate a new question on a skill selected by the student's degree of knowledge in the various skills
        of the carrier path. 
        The difficulty of the question is balanced on the student's knowledge of the skill
        """
        welcome = self.models["question_generator"]([
            SystemMessage(
                content=f"act as a specialist in creating short and formal welcome messages in the context of assessment sessions for the {carrier_path_name} carrier path"),
            HumanMessage(
                content=f"creates a welcome message to a new assessment session in Python Developer for the user named John Smith"),
            SystemMessage(
                content=f"Dear John Smith,\n\nWelcome to your personalized Python Developer assessment session. We are thrilled to have you embark on this exciting path of enhancing your skills in Python development! \n\nThroughout this assessment, you will be presented with a series of tasks and questions designed to evaluate your understanding and proficiency in Python. The experience will be both challenging and rewarding, as it will provide you with the opportunity to showcase your talent while identifying areas for growth. Remember to answer short and concisely but be careful to create correct and complete answers. Good studying and... good luck with your test! Let's begin!"
            ),
            HumanMessage(
                content=f"creates a welcome message to a new assessment session for a role of Machine Learning Engineer for the user named TheBestInTheWorld"),
            SystemMessage(
                content=f"Hi TheBestInTheWorld,\n\nWelcome to your new assessment session in Machine Learning. This session is designed to assess your skills and knowledge in the field of Machine Learning exploring his algorithms, methodologies and models. \n\nRemember, this assessment is but a stepping stone in your learning journey. We wish you all the best and are excited to see your progress.\n\nUse short answers, if useful give some examples but still pay attention to answer completely.\n\nWarm Regards... let's start with the first question!"
            ),
            HumanMessage(
                content=f"creates a welcome message to a new assessment session in Data Scientist for the user named Mario"),
            SystemMessage(
                content=f"Welcome Mario,\n\nWe are happy to welcome you to your assessment session on the topics a Data Scientist should know well. This is an opportunity to demonstrate your skills and growth in the field of Data Science and in particular for the profession of Data Scientist.\n\nRemember that it is preferable to use short but always complete answers to the questions provided\n\nWe wish you the best in your rating and we look forward to seeing your solutions.\n\nLet's get things started with the first question... have fun!"
            ),
            HumanMessage(
                content=f"creates a welcome message to a new assessment session in {carrier_path_name} for the user named {student_name}"),
        ]).content
        question = self.models["question_generator"]([
            SystemMessage(
                content=f"act as an expert {carrier_path_name} instructor who has expertise in {area_name}, particularly regarding {skill.name}"),
            HumanMessage(
                content=f"write a very short question to evaluate my skills in {skill.name}; the difficulty of the question must be {skill.score} on a scale from 0 to 100; the question must be formulated in such a way as to allow a short textual answer to be able to evaluate my skills in {skill.name} write only the question, nothing else")
        ]).content
        return welcome, question

    def translate(self, message, language_from="italian", language_to="english"):
        return openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert translator from " + language_from + " to " + language_to + " with experience in texts related to the " + self.carrier_path.name + " .\nTranslate the user input into " + language_to + " and output only the translated text."
                },
                {
                    "role": "user",
                    "content": message
                }
            ],
            temperature=0,
            max_tokens=512,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        ).choices[-1].message.content
    
    def print_report(self, skills_list, level=1):
        for i in range(len(skills_list)):
            print('\t' * level, skills_list[i].name, '(' + str(skills_list[i].score) + ')')
            for o in range(len(skills_list[i].sub_skills)):
                self.print_report([skills_list[i].sub_skills[o]], level=level + 1)

    def step(self):
        if self.assessment_session_started:
            time_elapsed = int((datetime.now() - self.session_ts).total_seconds() / 60)
            if time_elapsed > SESSION_TIME_LIMIT:
                self.print_report(self.carrier_path.modules)
            skill_to_evaluate = self.get_skill_to_evaluate()
            
        else:
            self.assessment_session_started = True
            skill_to_evaluate = self.get_skill_to_evaluate()
            welcome, question = self.get_question(self.student_name, self.carrier_path.name,
                                                  skill_to_evaluate["area"].name, skill_to_evaluate["skill"])
            return welcome + "\n\n" + question

In [69]:
a = AISkiller()

In [60]:
from skills_eng import * 

__test = a.get_skill_to_evaluate()
print(a.carrier_path.name, '-', __test["area"].name, '-', __test["skill"].name)

Python Developer - Programming - DB Design


In [57]:
SKILL_OOPProgramming.score = 10
print(a.carrier_path.name, SKILL_Programming.name, SKILL_OOPProgramming.name)
print(a.get_question('Ciccio', a.carrier_path.name, SKILL_Programming.name, SKILL_OOPProgramming))

Python Developer Programming Object-Oriented Programming
("Dear Ciccio,\n\nWelcome to your tailored Python Developer assessment session! We are thrilled that you have chosen to take part in this exciting journey towards sharpening your Python development skills.\n\nDuring this session, you'll face various tasks and questions aimed at assessing your knowledge and competencies in Python. While the journey may be challenging, it's a fantastic opportunity for you to showcase your abilities and pinpoint areas for improvement. Remember, clear and concise responses are encouraged, but ensure they are entirely correct and complete.\n\nBest of luck with your assessment, and let's get started!", 'What is the concept of inheritance in Object-Oriented Programming?')


In [61]:
a.step()

"Dear Mario,\n\nWelcome to your Python Developer assessment session. We are delighted to accompany you on this journey of honing your Python development skills!\n\nIn this assessment, you will encounter a variety of questions and tasks tailored to evaluate your understanding and proficiency in Python programming. We believe this will be a fulfilling and enlightening experience for you.\n\nRemember, concise and precise answers are preferred. However, ensure your responses are complete and accurate. \n\nBest of luck with your assessment. Let's get started with the first question!\n\nWarm Regards.\n\nWhat is Python?"

In [70]:
a.print_report(a.carrier_path.modules)

	 Programming (0)
		 Procedural Programming (0)
		 Object-Oriented Programming (10)
		 Functional Programming (0)
		 Concurrent Programming (0)
		 DB Design (0)
		 Relational Databases (0)
			 MySQL (0)
			 PostgreSQL (0)
		 Python (0)
	 Networks (0)
		 Cloud Computing (0)
			 Introduction to Cloud Computing (0)
	 Data Science (0)
		 Data Driven Mindset (0)
	 Machine Learning (0)
		 Introduction to Machine Learning (0)
